# Topicos de estudo para a AI

## Fases de um projeto de machine learning versus etapas do CRISP-DM

<img src="crisp.png" width=80%/>

## Análise Exploratória

- O que fazer **antes** da separação treino-teste e o que fazer **depois**

    - Antes: analise da natureza global das variaveis, independentemente uma das outras

        - Só coisa "ingênua", que não leva diretamente a construção de modelos

        - Análise de anomalias: valores faltantes, outliers, erros grosseiros

        - Variáveis contínuas:
        
            - Medidas descritivas (média, mediana, desvio padrão, etc)

            - Histogramas

        - Variáveis categóricas:

            - Frequências (`value_counts`)

    - Separação treino-teste:

        - Nada especial, só lembre de fixar o `random_state`

            > ```Python
            > 
            > from sklearn.model_selection import train_test_split
            > 
            > SEED = 42
            > 
            > X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
            > 
            > ```

    - Depois: tá liberado, mas só no conjunto de treino

        - Analisar dependencias entre variaveis, e entre elas e o target

        - Contínua versus contínua:

            - Medidas de correlação (Pearson, Spearman, Kendall)

            - Gráficos de espalhamento (*scatter plots*)

        - Categórica versus categórica

            - Teste de independencia: teste qui-quadrado de Pearson (*chi-square*). **Não foi visto em aula, não se preocupe.**

            - Tabela de contingência (`cross_tab`)

        - Categórica versus contínua

            - Boxplot da contínua para cada categoria

            - Teste de Kolmogorov-Smirnov entre valores da variável contínua para pares de categorias. **Não foi visto em aula, não se preocupe.**


In [10]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)
print(data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

:Number of Instances: 178
:Number of Attributes: 13 numeric, predictive attributes and the class
:Attribute Information:
    - Alcohol
    - Malic acid
    - Ash
    - Alcalinity of ash
    - Magnesium
    - Total phenols
    - Flavanoids
    - Nonflavanoid phenols
    - Proanthocyanins
    - Color intensity
    - Hue
    - OD280/OD315 of diluted wines
    - Proline
    - class:
        - class_0
        - class_1
        - class_2

:Summary Statistics:

============================= ==== ===== ======= =====
                                Min   Max   Mean     SD
============================= ==== ===== ======= =====
Alcohol:                      11.0  14.8    13.0   0.8
Malic Acid:                   0.74  5.80    2.34  1.12
Ash:                          1.36  3.23    2.36  0.27
Alcalinity of Ash:            10.6  30.0    19.5   3.3
Magnesium:                    70.0 162.0    99.7  14.3

In [15]:
dataset = data.data
dataset['target'] = data.target

dataset

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [16]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       178 non-null    float64
 1   malic_acid                    178 non-null    float64
 2   ash                           178 non-null    float64
 3   alcalinity_of_ash             178 non-null    float64
 4   magnesium                     178 non-null    float64
 5   total_phenols                 178 non-null    float64
 6   flavanoids                    178 non-null    float64
 7   nonflavanoid_phenols          178 non-null    float64
 8   proanthocyanins               178 non-null    float64
 9   color_intensity               178 non-null    float64
 10  hue                           178 non-null    float64
 11  od280/od315_of_diluted_wines  178 non-null    float64
 12  proline                       178 non-null    float64
 13  targe

In [17]:
dataset.select_dtypes(include='float64').describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


## Pipelines

### Transformadores

<img src="transformadores.png" width=70%/>

### Pipelines de transformação de dados

<img src="pipeline.png" width=50%/>

Para quem olha "de fora", uma *pipeline* se parece com um transformador e nada mais! Podemos treiná-lo com o método `fit`:

<img src="pipeline_transform_train.png" width=50%/>

E o que acontece "por trás das cortinas"? Deixa que o *Scikit-Learn* cuida de tudo para você! Mas, para nosso entendimento, eis o que acontece:

<img src="pipeline_transform_train_internals.png" width=50%/>

Agora que o pipeline está treinado, podemos usá-lo para fazer transformações de dados, como um transformer qualquer!

<img src="pipeline_transform_apply.png" width=50%>

E, por dentro, o que acontece? O *Scikit-Learn* vai chamar cada transformador, um por vez, em sequência:

<img src="pipeline_transform_apply_internals.png" width=50%/>

### Pipelines preditoras

Toda *pipeline* é feita de uma sequência de estágios *transformadores*, e o último estágio pode ser um *transformador* ou um *modelo preditivo*.

- ***Pipeline*** **transformadora**:

Quando o último estágio é um *transformador* (como no exemplo anterior), a *pipeline* atua como um *transformador*

- ***Pipeline*** **preditora**:

Quando o último estágio é um *modelo preditivo*, a *pipeline* atua como um *modelo preditivo*

<img src="pipeline_predict.png" width=50%/> 

Treinamos a *pipeline* preditora do mesmo jeito que treinamos modelos:

<img src="pipeline_predict_train.png" width=50%>

"Por trás dos panos":

<img src="pipeline_predict_train_internals.png" width=80%>

Para fazer predições, use como se fosse um modelo qualquer!

<img src="pipeline_predict_apply.png" width=50%/>

Novamente, o que acontece "por dentro":

<img src="pipeline_predict_apply_internals.png" width=90%/>

### `ColumnTransformer`, `Pipeline` e transformadores

> ```Python
> geo_cols = [
>     'longitude',
>     'latitude',
> ]
> 
> numerical_cols = [
>     'housing_median_age',
>     'log_households',
>     'log_median_income',
>     'log_rooms_per_household',
>     'log_population_per_household',
>     'log_bedrooms_per_room',
> ]
> 
> categorical_cols = [
>     'ocean_proximity',
> ]
> 
> geo_pipeline = Pipeline([
>     ('imputer', SimpleImputer(strategy='median')),
>     ('cluster', KMeans(n_clusters=50)),
> ])
> 
> num_pipeline = Pipeline([
>     ('imputer', SimpleImputer(strategy='median')),
>     ('poly', PolynomialFeatures(degree=3, include_bias=False)),
>     ('scaler', StandardScaler()),
> ])
> 
> cat_pipeline = Pipeline([
>     ('encoder', OneHotEncoder(sparse_output=False)),
> ])
> 
> preprocessing_pipe = ColumnTransformer(
>     transformers=[
>         ('geo', geo_pipeline, geo_cols),
>         ('num', num_pipeline, numerical_cols),
>         ('cat', cat_pipeline, categorical_cols),
>     ],
>     remainder='passthrough',
> )
> ```

<img src="column_transformer.png" width=100%/>

## Comparação de modelos

### Versão simples: train-test-val split

### Versão mais sofisticada: validação cruzada

## Ajuste de hiperparâmetros

- `GridSearchCV`

## Regressão

- Medidas de desempenho: MSE, RMSE

- Análise de erros:

    - Resíduos

- Modelo trivial

## Classificação

- Classificação binária e multiclasse

    - Classificador trivial

De resto, só classificação binária.

- Medidas de desempenho:

    - acurácia

    - Precision e recall

        - trade-off precision vs. recall

        - Curva precision-recall, `cross_val_predict`

        - métrica F1

    - Sensibility e specificity

        - trade-off sensibility vs. specificity

        - Curva ROC, área sob a curva ROC (AUC ou AUROC)

## Modelo linear

- Definição

- Explicação intuitiva de porque não podemos ter *features colineares*